# Web-Project

## Propósito del proyecto

El principal propósito de este proyecto es obtener una lista de citas célebres, junto con su autor y con las caractéristicas de la cita, como el tipo de cita que es. Esta información la obtendres haciedno web scraping de la web "https://www.azquotes.com/top_quotes.html", la cual contiene exactamente 1000 citas célebres.

A partir de ahí la idea es utilizar la API de Wikipedia para obtener información respecto al autor de cada cita. Y una vez la haya conseguido, juntar ambas bases de datos para hacer una más grande y enriquecida.

## Código

A continuación importo las librerías que vaya considerando necesarias para trabajar con la extracción de los datos.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from lxml import html
import urllib.request
from urllib.request import urlopen
import re
import wikipedia
import itertools
import json
import tweepy
from translate import Translator

# from py_translator import Translator
# from googletrans import Translator
# from pprint import pprint
# from lxml.html import fromstring
# import random
# import scrapy

## Web scraping

En la celda de debajo he escrito el código con el que obtengo las citas con sus respectivo autores y tags, rotando por cada una de las 10 páginas de la web a través del loop for. Una vez tengo toda la información que quiero, la junto utilizando un diccionario, y la convierto en un DataFrame de 4 columnas, incluyendo el índice, el cual he formateado para que empiece por 1, en vez de 0.

In [2]:
list_of_quotes = []
list_of_authors = []
list_of_tags = []
url = 'https://www.azquotes.com/top_quotes.html?p=%s'

for i in range(1, 11):
    html = requests.get(url % i).content
    soup = BeautifulSoup(html, 'lxml')

    list_of_quotes.append([element.text.replace("\n", "") for element in soup.find_all('a',{'class':'title'})])
    merged_quotes = list(itertools.chain.from_iterable(list_of_quotes))
        
    list_of_authors.append([element.text.replace("\n", "").replace("Xunzi", "Xun Kuang").replace("John Lubbock", "John Lubbock, 1st Baron Avebury").replace("Drake", "Drake (musician)").replace("Charles Simmons", "Charles Simmons (author)").replace("James Allen", "James Allen (author)").replace("John Green", "John Green (author)").replace("Unknown", "Unknown (mathematics)") for element in soup.find_all('div',{'class':'author'})])
    merged_authors = list(itertools.chain.from_iterable(list_of_authors))
        
    list_of_tags.append([element.text.replace("\n", "") for element in soup.find_all('div',{'class':'tags'})])
    merged_tags = list(itertools.chain.from_iterable(list_of_tags))
        
data = pd.DataFrame({'Quote':merged_quotes,'Author':merged_authors,'Tags':merged_tags})
data.index = np.arange(1, len(data)+1)


## Wikipedia API

A continuación, utilizo la API de Wikipedia para ir rotando por la lista de autores de cada una de las citas para obtener un breve resumen con la biografía del autor y el link con el resto de la información en Wikipedia. Según sacamos la información, la incluimos en una lista que posteriormente convertiremos a DataFrame.

In [3]:
S = requests.Session()
URL = "https://en.wikipedia.org/w/api.php"

info = []
info2 = []
for author in merged_authors:
    PARAMS = {
        'action':"opensearch",
        'search':author,
        'limit': 0,
        'namespace':0,
        'profile':"normal",
        'format':"json"
        }
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    DATA2 = ''.join(DATA[2])
    DATA3 = ''.join(DATA[3])
    info.append(DATA2)
    info2.append(DATA3)
    
        
info = pd.DataFrame({'Authors_info':info, 'Wikipedia_link':info2})
info.index = np.arange(1, len(info)+1)

Aunque también encontré una seguna forma de hacerlo a través de comandos, que daba un resumen del autor un poco más largo en el que puedo especificar el número de frases a incluir, pero que no aportaba el link que finalmente también he incluido en la tabla.

In [4]:
#def quote(url):
#    list_of_quotes = []
#    list_of_authors = []
#    list_of_tags = []
#    authors_info = []
#    for i in range(1, 11):
#        html = requests.get(url % i).content
#        soup = BeautifulSoup(html, 'lxml')
#
#        list_of_quotes.append([element.text.replace("\n", "") for element in soup.find_all('a',{'class':'title'})])
#        merged_quotes = list(itertools.chain.from_iterable(list_of_quotes))
#        
#        list_of_authors.append([element.text.replace("\n", "").replace("Xunzi", "Xun Kuang").replace("John Lubbock", "John Lubbock, 1st Baron Avebury").replace("Drake", "Drake (musician)").replace("Charles Simmons", "Charles Simmons (author)").replace("James Allen", "James Allen (author)").replace("John Green", "John Green (author)").replace("Unknown", "Unknown (mathematics)") for element in soup.find_all('div',{'class':'author'})])
#        merged_authors = list(itertools.chain.from_iterable(list_of_authors))
#        
#        list_of_tags.append([element.text.replace("\n", "") for element in soup.find_all('div',{'class':'tags'})])
#        merged_tags = list(itertools.chain.from_iterable(list_of_tags))
#    
#    for author in merged_authors:
#        authors_info.append(wikipedia.summary(author, sentences=3))
#        
#    data = pd.DataFrame({'Quote':merged_quotes,'Author':merged_authors,'Tags':merged_tags,'Author_info':authors_info})
#    data.index = np.arange(1, len(data)+1)
#
#    return data
#    
#data = quote('https://www.azquotes.com/top_quotes.html?p=%s')

Ahora juntamos ambos datasets para crear una base de datos más grande y más enriquecida. Como podemos observar, la columna de la derecha corresponde con la información del autor extraida de la Wikipedia.

In [5]:
final = pd.concat([data, info], axis = 1, )
final.head()

,Quote,Author,Tags,Authors_info,Wikipedia_link
1,The essence of strategy is choosing what not t...,Michael Porter,"Essence, Deep Thought, Transcendentalism","Michael Eugene Porter (born May 23, 1947) is ...",https://en.wikipedia.org/wiki/Michael_Porter
2,Death is something inevitable. When a man has ...,Nelson Mandela,"Inspirational, Motivational, Death",Nelson Rolihlahla Mandela (; Xhosa: [xolíɬaɬa ...,https://en.wikipedia.org/wiki/Nelson_Mandela
3,"With or without religion, you would have good ...",Steven Weinberg,"God, Religious, Atheist","Steven Weinberg (; born May 3, 1933) is an Am...",https://en.wikipedia.org/wiki/Steven_Weinberg
4,A person who never made a mistake never tried ...,Albert Einstein,"Inspirational, Motivational, Change",Albert Einstein (; German: [ˈalbɛɐ̯t ˈʔaɪnʃtaɪ...,https://en.wikipedia.org/wiki/Albert_Einstein
5,"My mission in life is not merely to survive, b...",Maya Angelou,"Inspirational, Life, Inspiring",Maya Angelou ( (listen); born Marguerite Annie...,https://en.wikipedia.org/wiki/Maya_Angelou


## Twitter API

He decidido utilizar una segunda API para saber cómo de populares son en la actualidad los autores de las citas, obteniendo tweets clasificados como populares que contengan el nombre del autor. Una cosa importante a la hora de trabajar con esta API es que sólo puedes hacer 180 peticiones cada 15 minutos, por lo que he tenido que hacer 6 peticiones diferentes cada 15 minutos para buscar todos los autores.

In [9]:
API_KEY = "XXXX"
API_SECRET = "XXXX"
ACCESS_TOKEN = "XXXX"
ACCESS_TOKEN_SECRET = "XXXX"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [10]:
merged_authors180 = merged_authors[:180]
lista_tweets1 = []

for author in merged_authors180:
    results = api.search(q=author, result_type = "popular", count = 100)
    lista_tweets1.append(len(results))

In [13]:
merged_authors360 = merged_authors[180:360]
lista_tweets2 = []

for author in merged_authors360:
    results2 = api.search(q=author, result_type = "popular", count = 100)
    lista_tweets2.append(len(results2))

In [17]:
merged_authors540 = merged_authors[360:540]
lista_tweets3 = []

for author in merged_authors540:
    results3 = api.search(q=author, result_type = "popular", count = 100)
    lista_tweets3.append(len(results3))

In [21]:
merged_authors720 = merged_authors[540:720]
lista_tweets4 = []

for author in merged_authors720:
    results4 = api.search(q=author, result_type = "popular", count = 100)
    lista_tweets4.append(len(results4))

In [22]:
merged_authors900 = merged_authors[720:900]
lista_tweets5 = []

for author in merged_authors900:
    results5 = api.search(q=author, result_type = "popular", count = 100)
    lista_tweets5.append(len(results5))

In [24]:
merged_authors1000 = merged_authors[900:1000]
lista_tweets6 = []

for author in merged_authors1000:
    results6 = api.search(q=author, result_type = "popular", count = 100)
    lista_tweets6.append(len(results6))

Una vez he hecho las 6 peticiones, uno las 6 listas resultantes, confirmo que son 1000 registros los que hay, las convierto en DataFrame y las uno a la tabla ya existente resultando en una base de datos más enriquecida.

In [25]:
lista_tweets = lista_tweets1 + lista_tweets2 + lista_tweets3 + lista_tweets4 + lista_tweets5 + lista_tweets6

In [26]:
print(len(lista_tweets))

1000


In [27]:
lista_tweets = pd.DataFrame({'Popular_tweets_count':lista_tweets})
lista_tweets.index = np.arange(1, len(lista_tweets)+1)
lista_tweets.head()

,Popular_tweets_count
1,1
2,15
3,0
4,15
5,15


In [30]:
final2 = pd.concat([final, lista_tweets], axis = 1, )
final2.head()

,Quote,Author,Tags,Authors_info,Wikipedia_link,Popular_tweets_count
1,The essence of strategy is choosing what not t...,Michael Porter,"Essence, Deep Thought, Transcendentalism","Michael Eugene Porter (born May 23, 1947) is ...",https://en.wikipedia.org/wiki/Michael_Porter,1
2,Death is something inevitable. When a man has ...,Nelson Mandela,"Inspirational, Motivational, Death",Nelson Rolihlahla Mandela (; Xhosa: [xolíɬaɬa ...,https://en.wikipedia.org/wiki/Nelson_Mandela,15
3,"With or without religion, you would have good ...",Steven Weinberg,"God, Religious, Atheist","Steven Weinberg (; born May 3, 1933) is an Am...",https://en.wikipedia.org/wiki/Steven_Weinberg,0
4,A person who never made a mistake never tried ...,Albert Einstein,"Inspirational, Motivational, Change",Albert Einstein (; German: [ˈalbɛɐ̯t ˈʔaɪnʃtaɪ...,https://en.wikipedia.org/wiki/Albert_Einstein,15
5,"My mission in life is not merely to survive, b...",Maya Angelou,"Inspirational, Life, Inspiring",Maya Angelou ( (listen); born Marguerite Annie...,https://en.wikipedia.org/wiki/Maya_Angelou,15


## Translate API

Con la siguiente API (https://pypi.org/project/translate/) lo que pretendo es traducir cada una de las citas al español. Aunque el funcionamiento de esta API se basa en funciones y no en peticiones que se hacen a una URL tal como en el caso de la Wikipedia. Esta API, al igual que la anterior, tiene un límite de uso.

Para ello, y de forma similar a la anterior API, trabajo con un bucle que para cada cita me haga la traducción del inglés al español. Y como antes, inserto el resultado en un DataFrame que posteriormente fusiono con las dos bases de datos anteriores.

In [6]:
tranlations_list = []
for quote in merged_quotes:
    translator= Translator(to_lang="es")
    translation = translator.translate(quote)
    tranlations_list.append(translation)


In [8]:
tranlations_list = pd.DataFrame({'Quote_in_spanish':tranlations_list})
tranlations_list.index = np.arange(1, len(tranlations_list)+1)
tranlations_list.head()

,Quote_in_spanish
1,La esencia de la estrategia es elegir qué no h...
2,La muerte es algo inevitable. Cuando un hombre...
3,"Con o sin religión, tendrías gente buena hacie..."
4,Una persona que nunca cometió un error nunca p...
5,Mi misión en la vida no es meramente sobrevivi...


In [31]:
final3 = pd.concat([final2, tranlations_list], axis = 1, )
final3.head()

,Quote,Author,Tags,Authors_info,Wikipedia_link,Popular_tweets_count,Quote_in_spanish
1,The essence of strategy is choosing what not t...,Michael Porter,"Essence, Deep Thought, Transcendentalism","Michael Eugene Porter (born May 23, 1947) is ...",https://en.wikipedia.org/wiki/Michael_Porter,1,La esencia de la estrategia es elegir qué no h...
2,Death is something inevitable. When a man has ...,Nelson Mandela,"Inspirational, Motivational, Death",Nelson Rolihlahla Mandela (; Xhosa: [xolíɬaɬa ...,https://en.wikipedia.org/wiki/Nelson_Mandela,15,La muerte es algo inevitable. Cuando un hombre...
3,"With or without religion, you would have good ...",Steven Weinberg,"God, Religious, Atheist","Steven Weinberg (; born May 3, 1933) is an Am...",https://en.wikipedia.org/wiki/Steven_Weinberg,0,"Con o sin religión, tendrías gente buena hacie..."
4,A person who never made a mistake never tried ...,Albert Einstein,"Inspirational, Motivational, Change",Albert Einstein (; German: [ˈalbɛɐ̯t ˈʔaɪnʃtaɪ...,https://en.wikipedia.org/wiki/Albert_Einstein,15,Una persona que nunca cometió un error nunca p...
5,"My mission in life is not merely to survive, b...",Maya Angelou,"Inspirational, Life, Inspiring",Maya Angelou ( (listen); born Marguerite Annie...,https://en.wikipedia.org/wiki/Maya_Angelou,15,Mi misión en la vida no es meramente sobrevivi...


Por último, exporto la base de datos como archivo csv.

In [33]:
output = final.to_csv('output.csv', index=False)